In [3]:
# -*- coding: UTF-8 -*-
from PIL import Image
from tabula import read_pdf
import pandas as pd
from PIL import Image, ImageDraw, ImageFont

name_list = []
df = read_pdf('xiamen.pdf', pages="all", encoding='gb18030') 
for i in df.index:
    if i == 0:
        pass
    else:
        last_name = str(df.iat[i, 1]).strip().replace('?', '') #去掉两边空格，去掉中间?号，转成字符串
        first_name = str(df.iat[i, 2]).strip().replace(' ', '').replace('?', '') # 去掉两边和中间空格，去掉?号，转成字符串
        if "nan" not in last_name and "AWA" not in first_name: # 去掉无效名字
            #print(last_name, first_name)
            full_name = last_name + " " + first_name
            name_list.append(full_name) # 生成名单列表

name_string = " - ".join(name_list) # 将名单列表的名字用 - 连接生成字符串

ironman_image = Image.open("ironman.png").convert("L")
w, h = ironman_image.size
w = w * 3 #放大后的宽度（像素）
h = h * 3 #放大后的高度（像素）
ironman_image = ironman_image.resize((w, h)) # 等比放大
ironman_image_pixel = {} # 获取Ironman Logo的像素信息，每一行0代表白色像素，1代表灰色像素
for y in range(h):
    ironman_image_pixel[y] = []
    for x in range(w):
        pixel = ironman_image.getpixel((x, y))
        if pixel > 200: # 白色像素用0表示
            ironman_image_pixel[y].append(0)
        else:  # 灰色像素用1表示
            ironman_image_pixel[y].append(1)


# make a blank image for the text, initialized to transparent text color
iron_names = Image.new('RGBA', ironman_image.size, (255,255,255,0)) # 透明画布
# get a font
fnt_size = 20
fnt = ImageFont.truetype('C:/Windows/Fonts/Dengb.ttf', fnt_size)
# get a drawing context
iron_name_draw = ImageDraw.Draw(iron_names)

pix = 0
for y in range(h):
    if y % fnt_size == 0: # 在符合字体大小的行上才生成名字
        start_x = 0
        for dot, i in zip(ironman_image_pixel[y], range(w)):
            if (i+1)%(start_x+1) == 0: # 确保行中字母不重叠
                if dot == 0: # 图片白色部分跳过
                    start_x += fnt_size
                else:
                    if pix > len(name_string)-1: # 如果名单太短则从头来过
                        pix = 0
                    iron_name_draw.text((start_x,y), name_string[pix], font=fnt, fill=(255, 0, 0,255))# 生成不透明的红字
                    sw, sh = fnt.getsize(name_string[pix]) # 得到当前字母的像素宽度和高度
                    #sw1, sh1 = fnt.getsize(name_string[pix+1]) # 得到下一个字母的像素宽度和高度
                    start_x = start_x + sw # 确定下一个字母的起始x坐标
                    pix += 1
                    #if (start_x + sw1) > int(w):
                    #    break

iron_names.save("D:/ironman_name_list.png")
iron_names.show()

